In [ ]:
import pandas as pd
import numpy as np
!pip3 install transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

     |████████████████████████████████| 4.2 MB 33.7 MB/s 
     |████████████████████████████████| 84 kB 4.2 MB/s 
     |████████████████████████████████| 6.6 MB 64.0 MB/s 
     |████████████████████████████████| 596 kB 68.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
df = pd.read_csv("testset.csv")

In [ ]:
df

,sentence,report section,stock name,label,year
0,We serve consumers through our online and phys...,Item 1. Business,AMZN,0,2021
1,We are guided by four principles: customer obs...,Item 1. Business,AMZN,0,2021
2,"In each of our segments, we serve our primary ...",Item 1. Business,AMZN,0,2021
3,"In addition, we provide services, such as adve...",Item 1. Business,AMZN,0,2021
4,"We seek to offer our customers low prices, fas...",Item 1. Business,AMZN,0,2021
...,...,...,...,...,...
502,"Moreover, as additional competitors enter the ...",Item 7. Management’s Discussion and Analysis o...,TSLA,0,2021
503,The German Umweltbundesamt has issued our subs...,Item 7. Management’s Discussion and Analysis o...,TSLA,-1,2021
504,We are currently focused on ramping production...,Item 7. Management’s Discussion and Analysis o...,TSLA,0,2021
505,The Company's management is responsible for th...,Item 8. Financial Statements and Supplementary...,TSLA,0,2021


In [ ]:
df=df[['sentence','label']]
df

,sentence,label
0,We serve consumers through our online and phys...,0
1,We are guided by four principles: customer obs...,0
2,"In each of our segments, we serve our primary ...",0
3,"In addition, we provide services, such as adve...",0
4,"We seek to offer our customers low prices, fas...",0
...,...,...
502,"Moreover, as additional competitors enter the ...",0
503,The German Umweltbundesamt has issued our subs...,-1
504,We are currently focused on ramping production...,0
505,The Company's management is responsible for th...,0


In [ ]:
df_array = np.array(df)
df_list = list(df_array[:,0])
df_list

['We serve consumers through our online and physical stores and focus on selection, price, and convenience.',
 'We are guided by four principles: customer obsession rather than competitor focus, passion for invention, commitment to operational excellence, and long-term thinking.',
 'In each of our segments, we serve our primary customer sets, consisting of consumers, sellers, developers, enterprises, content creators, advertisers, and employees.',
 'In addition, we provide services, such as advertising to sellers, vendors, publishers, authors, and others, through programs such as sponsored ads, display, and video advertising.',
 'We seek to offer our customers low prices, fast and free delivery, easy-to-use functionality, and timely customer service.',
 'Competition continues to intensify, including with the development of new business models and the entry of new and well-funded competitors, and as our competitors enter into business combinations or alliances and established companies 

In [ ]:
def get_df(df):
    df_array = np.array(df)
    df_list = list(df_array[:,0])
    
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
    inputs = tokenizer(df_list, padding = True, truncation = True, return_tensors='pt')
    outputs = model(**inputs)
    
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    model.config.id2label

    
    positive = predictions[:, 0].tolist()
    negative = predictions[:, 1].tolist()
    neutral = predictions[:, 2].tolist()

    table = {'headline':df_list,
             "positive":positive,
             "negative":negative, 
             "neutral":neutral}

    df2 = pd.DataFrame(table, columns = ["headline", "positive", "negative", "neutral"])
    df2['sentiment']=df2[['positive','negative','neutral']].idxmax(axis=1)
    df2['actual']=df['label'].values.tolist()
    df2['same ind'] = (df2['sentiment']==df2['actual']).astype(int)

    return df2

In [ ]:
splits=np.array_split(df, 4)
df_total=[]
C=0
for split in splits:
    chunk=get_df(split)
    df_total.append(chunk)
    C=C+1
    print(C,len(splits))
    
df_final=pd.concat(df_total)

1 4
2 4
3 4
4 4


In [ ]:
splits

[                                              sentence  label
 0    We serve consumers through our online and phys...      0
 1    We are guided by four principles: customer obs...      0
 2    In each of our segments, we serve our primary ...      0
 3    In addition, we provide services, such as adve...      0
 4    We seek to offer our customers low prices, fas...      0
 ..                                                 ...    ...
 122  The Company’s capital expenditures were $16.7 ...      0
 123  If there is an abrupt and substantial decline ...     -1
 124  AC+ offers additional coverage for instances o...      0
 125  The Company sells its products and resells thi...      0
 126  Many of the Company’s competitors that sell mo...     -1
 
 [127 rows x 2 columns],
                                               sentence  label
 127  The Company believes it offers superior innova...      1
 128  The Company also competes with illegitimate me...     -1
 129  There can be no assura

In [ ]:
df_final["sentiment mapped"]=df_final["sentiment"].apply(lambda x: 1 if x=="positive" else (-1 if x=="negative" else 0))
df_final['same ind'] = (df_final['sentiment mapped']==df_final['actual']).astype(int)
#df_final
df_final

,headline,positive,negative,neutral,sentiment,actual,same ind,sentiment mapped
0,We serve consumers through our online and phys...,0.057271,0.012381,0.930348,neutral,0,1,0
1,We are guided by four principles: customer obs...,0.135138,0.012900,0.851962,neutral,0,1,0
2,"In each of our segments, we serve our primary ...",0.048831,0.013289,0.937881,neutral,0,1,0
3,"In addition, we provide services, such as adve...",0.058198,0.013054,0.928748,neutral,0,1,0
4,"We seek to offer our customers low prices, fas...",0.338998,0.008488,0.652514,neutral,0,1,0
...,...,...,...,...,...,...,...,...
121,"Moreover, as additional competitors enter the ...",0.939371,0.012988,0.047641,positive,0,0,1
122,The German Umweltbundesamt has issued our subs...,0.021396,0.925261,0.053343,negative,-1,1,-1
123,We are currently focused on ramping production...,0.923740,0.009363,0.066897,positive,0,0,1
124,The Company's management is responsible for th...,0.095672,0.014146,0.890182,neutral,0,1,0


In [ ]:
print(df_final["same ind"].sum()/len(df_final))

0.7199211045364892


In [ ]:
neutral=df_final[df_final["actual"]==0]
print(neutral["same ind"].sum()/len(neutral))

0.6839080459770115


In [ ]:
positive=df_final[df_final["actual"]==1]
print(positive["same ind"].sum()/len(positive))

0.7849462365591398


In [ ]:
negative=df_final[df_final["actual"]==-1]
print(negative["same ind"].sum()/len(negative))

0.6802721088435374


In [ ]:
df_final.to_csv("testset_label.csv")

In [ ]:
df_final[["headline","sentiment mapped"]].to_csv("jpm_onlylabel.csv")